In [426]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')



Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [427]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


## Down-sample

In [509]:
sample_A, sample_B = em.down_sample(A, B, size=18000, y_param=2)
# Save the datasets because it changes every time we perform down-sampling.
em.to_csv_metadata(sample_A, '../datasets/sample_A_generated.csv')
em.to_csv_metadata(sample_B, '../datasets/sample_B_generated.csv')

0%                          100%
[##############################] | ETA: 00:00:24 | ETA: 00:00:22 | ETA: 00:00:21 | ETA: 00:00:20 | ETA: 00:00:19 | ETA: 00:00:18 | ETA: 00:00:17 | ETA: 00:00:17 | ETA: 00:00:16 | ETA: 00:00:15 | ETA: 00:00:14 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:24


True

## Blocking

In [ ]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

In [ ]:
# Describe the candidate set
print(len(D))
D.head(2)

In [ ]:
# Sample the candidate set.
S = em.sample_table(D, sample_size=1000) # C is the candidate set to be sampled from.
# Write the sample set to a file for further processing.
em.to_csv_metadata(S, '../datasets/candidate_sample_generated.csv')

## Feature extraction

In [498]:
# Loading the saved files
import py_entitymatching as em
sample_A = em.read_csv_metadata('../datasets/sample_A.csv')
sample_B = em.read_csv_metadata('../datasets/sample_B.csv')
S = em.read_csv_metadata('../datasets/candidate_sample.csv')
G = em.read_csv_metadata('../datasets/candidate_sample_labeled.csv')

# Adding metadata to the files
em.set_key(sample_A, 'ID')
em.set_key(sample_B, 'ID')

em.set_key(S, '_id')
em.set_fk_ltable(S, 'ltable_ID')
em.set_fk_rtable(S, 'rtable_ID')
em.set_ltable(S, sample_A)
em.set_rtable(S, sample_B)

em.set_key(G, '_id')
em.set_fk_ltable(G, 'ltable_ID')
em.set_fk_rtable(G, 'rtable_ID')
em.set_ltable(G, sample_A)
em.set_rtable(G, sample_B)
IJ = em.split_train_test(G, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [501]:
# Manual Feature Extraction
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
match_f = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)
features_to_drop = list(range(0, 31))
match_f = match_f.drop(features_to_drop)  # We drop all features, as we build features manually.

# Create blackbox features
import re
def custom_address_sim(ltuple, rtuple):
    sim = 0.0
    # Extract all numbers from the address and count how many of them match-
    numbers = re.findall(r'\d+', ltuple['address'])
    num_matches = 0
    for number in numbers:
        if number in rtuple['address']:
            num_matches = num_matches + 1
    if len(numbers) > 0:
        sim = float(num_matches) / float(len(numbers)) # Normalize    
    # Compute jaccard similarity for street names.
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['address']), em.tok_alphabetic(rtuple['address']))
    # Only addresses belonging to same zipcodes are relevant.
    zipcode_exact_match = em.exact_match(ltuple['zipcode'], rtuple['zipcode'])
    return sim * jaccard_sim * zipcode_exact_match

def custom_name_jaccard(ltuple, rtuple):
    jaccard_sim = em.jaccard(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return jaccard_sim

def custom_address_overlap(ltuple, rtuple):    
    sim = em.overlap_coeff(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    return sim * em.exact_match(ltuple['zipcode'], rtuple['zipcode'])

def custom_name_overlap(ltuple, rtuple):
    overlap_sim = em.overlap_coeff(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return overlap_sim

def custom_address_cosine(ltuple, rtuple):    
    sim = em.cosine(em.tok_alphanumeric(ltuple['address']), em.tok_alphanumeric(rtuple['address']))
    # Only addresses belonging to same zipcodes are relevant.
    zipcode_exact_match = em.exact_match(ltuple['zipcode'], rtuple['zipcode']) 
    return sim * zipcode_exact_match

def custom_name_cosine(ltuple, rtuple):
    sim = em.cosine(em.tok_alphabetic(ltuple['name']), em.tok_alphabetic(rtuple['name']))
    return sim

def custom_cuisine_overlap(ltuple, rtuple):
    sim = em.overlap_coeff(em.tok_alphabetic(ltuple['cuisine']), em.tok_alphabetic(rtuple['cuisine']))
    return sim

status = em.add_blackbox_feature(match_f, 'custom_address_sim', custom_address_sim)
status = em.add_blackbox_feature(match_f, 'custom_name_jaccard', custom_name_jaccard)
status = em.add_blackbox_feature(match_f, 'custom_name_overlap', custom_name_overlap)
status = em.add_blackbox_feature(match_f, 'custom_address_overlap', custom_address_overlap)
status = em.add_blackbox_feature(match_f, 'custom_name_cosine', custom_name_cosine)
status = em.add_blackbox_feature(match_f, 'custom_address_cosine', custom_address_cosine)
status = em.add_blackbox_feature(match_f, 'custom_cuisine_overlap', custom_cuisine_overlap)

match_f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
31,ID_ID_lev_sim,ID,ID,None,None,lev_sim,<function ID_ID_lev_sim at 0x16ce47488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,custom_address_sim,None,None,None,None,None,<function custom_address_sim at 0x16ce47268>,None,False
2,custom_name_jaccard,None,None,None,None,None,<function custom_name_jaccard at 0x16ce47e18>,None,False
3,custom_name_overlap,None,None,None,None,None,<function custom_name_overlap at 0x16ce47378>,None,False
4,custom_address_overlap,None,None,None,None,None,<function custom_address_overlap at 0x16ce47048>,None,False
5,custom_name_cosine,None,None,None,None,None,<function custom_name_cosine at 0x16ce472f0>,None,False
6,custom_address_cosine,None,None,None,None,None,<function custom_address_cosine at 0x16ce471e0>,None,False
7,custom_cuisine_overlap,None,None,None,None,None,<function custom_cuisine_overlap at 0x17cdf7730>,None,False


In [502]:
H = em.extract_feature_vecs(J, feature_table=match_f,attrs_after=['label'])
H.head(10)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_ID,rtable_ID,ID_ID_lev_sim,custom_address_sim,custom_name_jaccard,custom_name_overlap,custom_address_overlap,custom_name_cosine,custom_address_cosine,custom_cuisine_overlap,label
993,1212787,8272,13604,0.0,0.0,0.000000,0.000000,0.800000,0.000000,0.800000,0.0,0
859,1197994,8067,13143,0.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1
298,603817,2606,8117,0.0,0.4,1.000000,1.000000,1.000000,1.000000,0.755929,1.0,1
553,916195,4515,22908,0.0,0.0,0.000000,0.000000,0.833333,0.000000,0.833333,0.0,0
672,1061638,5857,12524,0.2,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1
971,1212418,8265,12869,0.4,0.0,0.000000,0.000000,0.800000,0.000000,0.800000,0.0,0
27,16580,58,10428,0.2,1.0,0.000000,0.000000,1.000000,0.000000,1.000000,1.0,0
231,553645,2365,15363,0.4,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1
306,651691,2726,9079,0.0,1.0,0.285714,0.666667,1.000000,0.471405,1.000000,1.0,1
706,1116640,6857,21344,0.0,1.0,0.250000,0.400000,1.000000,0.400000,1.000000,0.0,0


## Matching

In [503]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
nb = em.NBMatcher(name='NaiveBayes')
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

In [504]:
# Report Precision
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10a7e8be0>,5,0.978723,0.976744,0.964912,0.944444,0.926829,0.958331
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10a7e8ef0>,5,1.000000,0.977273,0.948276,0.972222,0.930233,0.965601
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10a7e8d30>,5,0.979167,0.955556,0.964912,0.972222,0.951220,0.964615
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10a7e8a90>,5,1.000000,0.977273,0.948276,0.972973,0.909091,0.961522
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10a7e8518>,5,1.000000,0.977273,0.947368,0.972973,0.909091,0.961341
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10a7e8b00>,5,1.000000,0.977273,0.965517,0.972973,0.930233,0.969199


In [505]:
# Report Recall
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10a7e8be0>,5,0.938776,0.893617,0.932203,0.894737,0.904762,0.912819
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10a7e8ef0>,5,0.959184,0.914894,0.932203,0.921053,0.952381,0.935943
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10a7e8d30>,5,0.959184,0.914894,0.932203,0.921053,0.928571,0.931181
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10a7e8a90>,5,0.938776,0.914894,0.932203,0.947368,0.952381,0.937124
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10a7e8518>,5,0.938776,0.914894,0.915254,0.947368,0.952381,0.933735
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10a7e8b00>,5,0.959184,0.914894,0.949153,0.947368,0.952381,0.944596


In [506]:
# Report F1
# Select the best ML matcher using CV
result = em.select_matcher([dt, nb, rf, svm, ln, lg], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x10a7e8be0>,5,0.958333,0.933333,0.948276,0.918919,0.915663,0.934905
1,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x10a7e8ef0>,5,0.979167,0.945055,0.940171,0.945946,0.941176,0.950303
2,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x10a7e8d30>,5,0.969072,0.934783,0.948276,0.945946,0.939759,0.947567
3,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x10a7e8a90>,5,0.968421,0.945055,0.940171,0.960000,0.930233,0.948776
4,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x10a7e8518>,5,0.968421,0.945055,0.931034,0.960000,0.930233,0.946949
5,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x10a7e8b00>,5,0.979167,0.945055,0.957265,0.960000,0.941176,0.956533


# Debug Matcher

In [507]:
# Train using feature vectors from I
G = em.extract_feature_vecs(I, feature_table=match_f, attrs_after='label')
dt = em.DTMatcher()
dt.fit(table=G, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')
pred_table = dt.predict(table=G,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],  append=True, target_attr='predicted')
eval_summary = em.eval_matches(pred_table, 'label', 'predicted')
false_pos_df = em.get_false_positives_as_df(G, eval_summary)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00


# Find Matcher after debugging

In [508]:
# Train using feature vectors from I
matchers = [dt, nb, rf, svm, ln, lg];
names = ['Decision Tree', 'Naive Bayes', 'Random Forest', 'SVM', 'Linear Regression', 'Logistic Regression']
for i in range(0,len(matchers)):
    matcher = matchers[i]
    name = names[i]
    matcher.fit(table=H, 
           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
           target_attr='label')

    # Convert J into a set of feature vectors using F
    L = em.extract_feature_vecs(J, feature_table=match_f,attrs_after=['label'])

    # Predict on L 
    predictions = matcher.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                  append=True, target_attr='predicted', inplace=False)
    # Evaluate the predictions
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    print(name)
    em.print_eval_summary(eval_result)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[##########                    ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Decision Tree
Precision : 99.56% (226/227)
Recall : 96.17% (226/235)
F1 : 97.84%
False positives : 1 (out of 227 positive predictions)
False negatives : 9 (out of 273 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[#########                     ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Naive Bayes
Precision : 96.49% (220/228)
Recall : 93.62% (220/235)
F1 : 95.03%
False positives : 8 (out of 228 positive predictions)
False negatives : 15 (out of 272 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[#########                     ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

...

SVM
Precision : 96.1% (222/231)
Recall : 94.47% (222/235)
F1 : 95.28%
False positives : 9 (out of 231 positive predictions)
False negatives : 13 (out of 269 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00
0%                          100%
[#########                     ] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

Linear Regression
Precision : 96.09% (221/230)
Recall : 94.04% (221/235)
F1 : 95.05%
False positives : 9 (out of 230 positive predictions)
False negatives : 14 (out of 270 negative predictions)


[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:00

Logistic Regression
Precision : 96.94% (222/229)
Recall : 94.47% (222/235)
F1 : 95.69%
False positives : 7 (out of 229 positive predictions)
False negatives : 13 (out of 271 negative predictions)
